In [1]:
# Always neds to be done in Rasberry Pi
import findspark
findspark.init('/home/baxman/spark-2.4.7-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_realistic').getOrCreate()

In [2]:
# Import LinReg from MLlib
from pyspark.ml.regression import LinearRegression

In [4]:
# Set training
data = spark.read.csv('/home/baxman/Codes/PySpark/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [12]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [14]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
# Setting up data for usage in MLlib

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [55]:
# Assemble vector of feature columns

assembler = VectorAssembler(inputCols=['Avg Session Length',
                                     'Time on App',
                                     'Time on Website',
                                     'Length of Membership',],
                           outputCol = 'features')

In [56]:
output = assembler.transform(data)

In [57]:
# Created a new feautures column
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [58]:
# Features is now a dense vector
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [59]:
# Make final data
final_data = output.select('features', 'Yearly Amount Spent')

In [60]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [61]:
# Train/test split
training_data, test_data = final_data.randomSplit([0.75,0.25])

In [62]:
training_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                359|
|   mean| 497.74921260028475|
| stddev|  80.39947260531135|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [63]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                141|
|   mean| 503.29823975739885|
| stddev|  76.61919944551398|
|    min|  282.4712457199145|
|    max|  744.2218671047146|
+-------+-------------------+



In [64]:
# Create lr model
lr = LinearRegression(labelCol = 'Yearly Amount Spent')


In [65]:
lr_model = lr.fit(training_data)

In [66]:
# Evaluate
test_results = lr_model.evaluate(test_data)

In [67]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.876741065908845|
| -4.726498234976191|
|  7.502102251176893|
|-20.684807853699454|
|-0.8381874028697212|
|-13.970941433549342|
| -1.747294916877422|
|   17.3626071507054|
| -4.810356289896674|
| -0.700359609056818|
|-2.7189665331486594|
| -1.583731254349857|
|  7.508788691121197|
|  8.681057762304874|
|-1.8277870569730794|
| -10.45839582887345|
|  5.034858851159811|
|-17.637577534920183|
|0.13023584227397578|
| -8.523513793290476|
+-------------------+
only showing top 20 rows



In [68]:
# Show RMSE
test_results.rootMeanSquaredError

10.571257473784842

In [69]:
# R2
test_results.r2

0.9808279232739074

In [70]:
# Final data comparison

final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [71]:
# Predict on unlabled data

unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.4925366965402...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.2606468698795...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.6610498227460...|
|[31.8124825597242...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.8627411090001...|
|[31.8648325480987...|
|[31.9480174211613...|
|[31.9563005605233...|
|[32.0047530203648...|
|[32.0085045178551...|
+--------------------+
only showing top 20 rows



In [72]:
predictions = lr_model.transform(unlabeled_data)

In [73]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|397.76361000671864|
|[30.4925366965402...| 287.1977439548907|
|[30.9716756438877...| 487.1365075057158|
|[31.1239743499119...| 507.6318616934652|
|[31.2606468698795...| 422.1648186598211|
|[31.5741380228732...| 558.3802135941362|
|[31.5761319713222...| 542.9738789062058|
|[31.6098395733896...|427.18294250040276|
|[31.6253601348306...|381.14725704682087|
|[31.6610498227460...|417.05871318895765|
|[31.8124825597242...| 395.5293115169459|
|[31.8186165667690...| 448.0024046244855|
|[31.8209982016720...|417.16649232209215|
|[31.8512531286083...| 464.3111889044935|
|[31.8627411090001...| 558.1259282310198|
|[31.8648325480987...| 450.3496763056871|
|[31.9480174211613...|  456.886018041738|
|[31.9563005605233...|  564.763509282119|
|[32.0047530203648...|463.61574527835546|
|[32.0085045178551...|451.72073482204587|
+--------------------+------------